In [1]:
import sklearn.svm as svm
import scipy as sp
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [2]:
clf = svm.SVC(kernel="linear", C=1)
clf2= svm.SVC(kernel="linear", C=1)
pipe = make_pipeline(StandardScaler(),clf)
X = np.array([[-1, -1], [-2, -1],[-3,-1],[3,1], [1, 1], [2, 1],[2.5,1],[1.5,1]])
Z = np.array([[-1, -1], [-2, -1], [1, 1], [2, 1]])
A = np.array([1, 1, 2,2])
Y = np.array([1, 1, 1, 1, 2, 2,2,2])
clf.fit(X,Y)
clf2.fit(Z,A)
R=np.array_split(X,3)
print(R)
# svm.Pipeline(steps=[('standardscaler', StandardScaler()),
#                 ('svc', SVC(gamma='auto'))])

print(clf.predict([[-.8,-1]]))
vecotr= clf.support_vectors_
vecotr2= clf2.support_vectors_
print(clf.decision_function(vecotr))
print(clf2.decision_function(vecotr2))
for i in vecotr:
    print(clf.decision_function([i]))
#print(clf.decision_function([vecotr]))
#print(clf.decision_function([vecotr2]))
marginDensity= 0

#initiializes drift detector
class DriftDetector:
    def __init__(self, data, labels,kernel,C,K):
        svm= DriftDetector.trainSVM(data,labels,kernel,C)
        self.svm= svm
        density = DriftDetector.findMarginDensity(data,svm)
        self.max=density
        self.min=density
        self.thresh=.075
        #needed for other version.
        avAc,stdAc,stdDens= DriftDetector.KXVal(data,labels,kernel,C,K)
        self.density=density
        self.avAc=avAc
        self.stdAc=stdAc
        self.stdDens=stdDens
        self.stdDensThresh=1

#needs to be adapted for window stuff.
    def findMarginDensity(batch,svm):
        #vector = svm.support_vectors_[:,0]
        marginDensity=0
        for vect in batch:
            if np.abs(svm.decision_function([vect]))<=1:
                marginDensity= marginDensity+1
        return marginDensity/batch.size

    def trainSVM(data,labels,kernel,C):
        clf= svm.SVC(kernel=kernel,C=C)
        clf.fit(data,labels)
        return clf
#1st version md3
    def detectDrift(self,batch):
        md2=self.findMarginDensity(batch)
        if md2>self.max:
            self.max=md2
        if md2<self.min:
            self.min= md2
        if (self.max-self.min)>self.thresh:
            return True
        else: False
    #multiclass? linear? etc. retrain once found problem.
    def retrain(self,data,labels,kernel,C,K):
        self.svm.fit(data,labels)
        density = self.findMarginDensity(data)
        self.max=density
        self.min=density
        avAc,stdAc,stdDens= DriftDetector.KXVal(data,labels,kernel,C,K)
        self.density=density
        self.avAc=avAc
        self.stdAc=stdAc
        self.stdDens=stdDens
##cross avlidaton, gets the distribution components.
    def KXVal(data, labels,kernel,C,K):
        splitData=np.array_split(data,K)
        splitLabels=np.array_split(labels,K)
        accuracies = np.zeros()
        densities = np.zeros
        for i in range(K):
            trainData=splitData[:i]+splitData[i:]
            trainLabels=splitLabels[:i]+splitLabels[i:]
            accuracies[i],densities[i]= DriftDetector.CrossValid(trainData,splitData[i],trainLabels,splitLabels[i],kernel,C)
        averageAccuracy= np.average(accuracies)
        standardDevAccuracy= np.std(accuracies)
        standardDevDensity=np.std(densities)
        return averageAccuracy, standardDevAccuracy,standardDevDensity

#this is for batch, not sliding window. sliding window forgetting factor sus
#here supposed to query an oracle callback function.
    def calculateMD3drift(self,batch):
        batchMD= DriftDetector.findMarginDensity(batch,self.svm)
        if np.abs(batchMD-self.density)>self.stdDensThresh*self.stdDens:
            return True
        else:
            return False

    def checkAccuracy(self,batch,labels):
        if batch.size==labels.size:
            score=self.svm.score(batch,labels)
            if self.avAc-score>self.stdDensThresh*self.stdAc:
                return True
            else:
                return False

            
    # very inefficient, right now, creates new svm. need change the svm, so need to use class svm and fit.
    def CrossValid(trainData,testData,trainLabels,testLabels,kernel,C):
        svm= DriftDetector.trainSVM(trainData,trainLabels,kernel,C)
        density = DriftDetector.findMarginDensity(trainData,svm)
        score= svm.score(testData,testLabels)
        return density, score




    #need k cross validation. upgraded
    #need the fuzzy algorithm -> basically same algo but using RS and fuzzy. majority voting.
    ##need virtual margins implementation. inda dumb, just about confidence levels?
    ##ask lorena: if the same but with different way to find classification, not ... if using window.
    #non lineear margins... vrtual.
    
    
dd= DriftDetector(X,Y,"linear",1)
if dd.detectDrift(X):
    print("yes")
else:
    print("no")




[array([[-1., -1.],
       [-2., -1.],
       [-3., -1.]]), array([[3., 1.],
       [1., 1.],
       [2., 1.]]), array([[2.5, 1. ],
       [1.5, 1. ]])]
[1]
[-0.99957031 -1.00035156  1.00042969  0.99964844  1.00003906  1.00023437]
[-1.  1.]
[-0.99957031]
[-1.00035156]
[1.00042969]
[0.99964844]
[1.00003906]
[1.00023437]
no
